In [5]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Flatten, Conv2D
from keras.preprocessing.image import ImageDataGenerator
import glob

In [6]:
img_height=80
img_width=80
channel=3
X_train=np.empty([8005, img_height, img_width, channel], np.int32)
X_test=np.empty([2023, img_height, img_width, channel], np.int32)
y_train=np.full([8005,], fill_value=5)
y_test=np.full([2023,], fill_value=6)
i = 0
# cat = 0 , dog =1

In [7]:
for filepath in glob.iglob('training_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 0
    X_train[i] = img
    i = i + 1
    
for filepath in glob.iglob('training_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 1
    X_train[i] = img
    i = i + 1    
    
i=0    
    
for filepath in glob.iglob('test_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 0
    X_test[i] = img
    i = i + 1
    
for filepath in glob.iglob('test_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 1
    X_test[i] = img
    i = i + 1      

In [10]:
X_train = X_train/255
X_test  = X_test/255
y_train = keras.utils.to_categorical(y_train, 2)
y_test  = keras.utils.to_categorical(y_test, 2)
input_shape=(img_height, img_width, channel)

In [22]:
img_aug = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.15,
                height_shift_range=0.15,
                horizontal_flip=True)

img_aug.fit(X_train, augment=True)

In [29]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit_generator(img_aug.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train) / 32, verbose=1, epochs=50) 

Epoch 1/50
251/250 [==============================] - 34s 135ms/step - loss: 0.6552 - accuracy: 0.6074
Epoch 2/50
251/250 [==============================] - 34s 137ms/step - loss: 0.5956 - accuracy: 0.6803
Epoch 3/50
251/250 [==============================] - 30s 118ms/step - loss: 0.5688 - accuracy: 0.7044
Epoch 4/50
251/250 [==============================] - 29s 117ms/step - loss: 0.5546 - accuracy: 0.7225
Epoch 5/50
251/250 [==============================] - 29s 116ms/step - loss: 0.5256 - accuracy: 0.7445
Epoch 6/50
251/250 [==============================] - 27s 110ms/step - loss: 0.5207 - accuracy: 0.7458
Epoch 7/50
251/250 [==============================] - 27s 109ms/step - loss: 0.5129 - accuracy: 0.7538
Epoch 8/50
251/250 [==============================] - 27s 109ms/step - loss: 0.5076 - accuracy: 0.7503
Epoch 9/50
251/250 [==============================] - 27s 109ms/step - loss: 0.4939 - accuracy: 0.7598
Epoch 10/50
251/250 [==============================] - 27s 108ms/step - l

In [31]:
score=model.evaluate(X_test, y_test)
print('Loss:', score[0])
print('Accuracy:',score[1])

2023/2023 [==============================] - 2s 902us/step
Loss: 0.37633990856956506
Accuracy: 0.8433020114898682


In [32]:
#model.save('cat_dog.h5')

This model achieved an accuracy of 84.33%. This is an improvement of almost 7% over my previous model which did not use image augmentation.